In [5]:
# PYTHON BINOMIAL OPTION PRICING
# Copyright Sheikh Pancham 2021 sheikh.pancham@gmail.com
# Connect on LinkedIn: https://www.linkedin.com/in/sheikhpancham/
# I, Sheikh Pancham, legally own this product/code and all intellectual property because I wrote it, and am therefore 
# authorized to distribute it freely to the global public. Any party is also authorized to freely distribute this 
# product/code but not to sell it. This product/code is being distributed as a research tool and there is no 
# guarantee of investment profits.

import numpy as np
import math as m
import scipy.stats as stats
from openpyxl import Workbook, load_workbook

class StockOption:
    def __init__(self):
        self._DeleteWorkbookSheets_() # To remove output sheets from previous runs
        self.CalcGreeks = 0

        # using openpyxl functionality
        self.wb_models = load_workbook('stockoptiondata.xlsx')
        self.ws_models = self.wb_models['Stock Option']
        self.S0 = self.ws_models['B4'].value
        self.K = self.ws_models['B5'].value
        self.r = self.ws_models['B6'].value
        self.div = self.ws_models['B7'].value
        self.T = self.ws_models['B8'].value
        self.sigma = self.ws_models['B9'].value
        self.n = max(1,self.ws_models['B10'].value) # Ensure n has at least 1 time step
        self.IsCall = self.ws_models['B9'].value
        self.IsEuropean = self.ws_models['B10'].value
             
    def _DeleteWorkbookSheets_(self): 
        # utility function to remove output sheets from previous runs
        wb = load_workbook('stockoptiondata.xlsx')
        ws = wb["Stock Option"]
        ws.freeze_panes = None
        ws_active = wb['Stock Option']
        # delete all sheets except Stock Option data sheet
        for sheet in wb.worksheets:
            if sheet.title != 'Stock Option':
                wb.remove(sheet) 
        
        for j in range(5,17):
            for i in range(5,9):
                ws.cell(i,j,0.0)
        for j in range(12,17):
            for i in range(12,15):
                ws.cell(i,j,0.0)
            
        wb.save("stockoptiondata.xlsx")
        
class BinomialOption(StockOption):
    def _InitParams_(self):
        self.dt = 0.0 # delta of time step, in years
        self.u = 0.0 # Up move increment
        self.d = 0.0 # Down move increment
        self.pu = 0.0 # Probability of Up move
        self.pd = 0.0 # Probability of Down move
    
    def _CRROptionPrice_(self):
        # Cox, Ross, Rubinstein (1979)
        self._InitParams_()
        self.dt = float(self.T)/float(self.n) # delta of time step, in years
        self.u = np.exp(self.sigma*(np.sqrt(self.dt)))
        self.d = 1 / self.u
        self.pu = (np.exp((self.r- self.div)*self.dt) - self.d) / (self.u - self.d)
        self.pd = 1 - self.pu
        self.model = 'CRR'

        if self.IsCall and self.IsEuropean:
            self.CRREuroCalloptionprice = round(self._CalcOptionPrice_(),4)
        elif not self.IsCall and self.IsEuropean:
            self.CRREuroPutoptionprice = round(self._CalcOptionPrice_(),4)
        elif self.IsCall and not self.IsEuropean:
            self.CRRAmerCalloptionprice = round(self._CalcOptionPrice_(),4)
        else:
            self.CRRAmerPutoptionprice = round(self._CalcOptionPrice_(),4)
        if self.CalcGreeks == 0:
            self._OutputModelTrees_()
        
    def _LROptionPrice_(self):
        # Leisen, Reimer (1996)
        self._InitParams_()
        temp_n = self.n
        if self.n % 2 == 0: self.n = self.n + 1
        self.dt = float(self.T)/float(self.n) # delta of time step, in years
    
        #Calculate LR up probability and down probability
        d1 = (np.log(self.S0 / self.K) + (((self.r - self.div) + 0.5 * self.sigma**2) * self.T)) \
                    / (self.sigma * np.sqrt(self.T)) # d1
        d2 = d1 - self.sigma * np.sqrt(self.T)
        
        signd1 = -1 if d1 < 0 else 1
        signd2 = -1 if d2 < 0 else 1
        
        # Peizer-Pratt Method 2 
        hzd1_exp_term = (d1 / (self.n + 1 / 3 + (0.1/(self.n+1))))**2 * (self.n + 1 / 6)
        hzd1 = 0.5 + signd1 / 2.0 * np.sqrt(1 - np.exp(-hzd1_exp_term)) 
        
        hzd2_exp_term = (d2 / (self.n + 1 / 3 + (0.1/(self.n+1))))**2 * (self.n + 1 / 6)
        hzd2 = 0.5 + signd2 / 2.0 * np.sqrt(1 - np.exp(-hzd2_exp_term))       
        
        pprime = hzd1
        self.pu = hzd2
        self.pd = 1 - self.pu
        
        self.u = np.exp((self.r-self.div) * self.dt) * pprime / self.pu
        self.d = np.exp((self.r-self.div) * self.dt) * (1 - pprime) / (1 - self.pu)
        self.model = 'LR'
        if self.IsCall and self.IsEuropean:
            self.LREuroCalloptionprice = round(self._CalcOptionPrice_(),4)
        elif not self.IsCall and self.IsEuropean:
            self.LREuroPutoptionprice = round(self._CalcOptionPrice_(),4)
        elif self.IsCall and not self.IsEuropean:
            self.LRAmerCalloptionprice = round(self._CalcOptionPrice_(),4)
        else:
            self.LRAmerPutoptionprice = round(self._CalcOptionPrice_(),4)
            
        if self.CalcGreeks == 0:
            self._OutputModelTrees_()
        self.n = temp_n
        
    def _JROptionPrice_(self):
        # Jarrow, Rudd 
        self._InitParams_()
        self.dt = float(self.T)/float(self.n) # delta of time step, in years
        self.pu = 0.5
        self.pd = 1 - self.pu 
        self.u = np.exp((self.r - self.div - self.sigma**2 * 0.5) * self.dt + self.sigma * np.sqrt(self.dt))
        self.d = np.exp((self.r - self.div - self.sigma**2 * 0.5) * self.dt - self.sigma * np.sqrt(self.dt))
        self.model = 'JR'
        
        if self.IsCall and self.IsEuropean:
            self.JREuroCalloptionprice = round(self._CalcOptionPrice_(),4)
        elif not self.IsCall and self.IsEuropean:
            self.JREuroPutoptionprice = round(self._CalcOptionPrice_(),4)
        elif self.IsCall and not self.IsEuropean:
            self.JRAmerCalloptionprice = round(self._CalcOptionPrice_(),4)
        else:
            self.JRAmerPutoptionprice = round(self._CalcOptionPrice_(),4)
        if self.CalcGreeks == 0:
            self._OutputModelTrees_()
    
    def _CalcOptionPrice_(self):
        self._StockPriceTree_()
        self._ProbabilityTree_()
        self._PayoffTree_()
        self._BackwardsDiscount_()
        return self.optionprice
    
    def _StockPriceTree_(self):
        self.StockTree = np.zeros((self.n+1,self.n+1))
        for j in range(self.n+1):
            for i in range(j+1):
                self.StockTree[i][j] = self.S0*(self.u**i)*(self.d**(j-i)) 

    def _ProbabilityTree_(self):
        self.ProbTree = np.zeros((self.n+1,self.n+1))
        for j in range(self.n+1):
            for i in range(j+1):
                self.ProbTree[i][j] = m.factorial(j) / (m.factorial(i) * m.factorial(j-i))\
                                                           * self.pu**i * self.pd**(j-i)
    
    def _PayoffTree_(self):
        if self.IsEuropean:
            self._EuropeanPayoffTree_()
        else:
            self._AmericanPayoffTree_()
        
    def _EuropeanPayoffTree_(self):
        #Initialize all payoff nodes to zeros
        self.PayoffTree = np.zeros((self.n+1,self.n+1))
        iopt = 0
        # Get payoffs at terminal nodes at the option maturity
        if self.IsCall:
            iopt = 1
        else:
            iopt = -1
        # Get payoffs at terminal nodes
        for i in range(self.n+1):
            self.PayoffTree[i][self.n] = np.maximum(0, iopt * (self.StockTree[i][self.n]-self.K))
            
    def _AmericanPayoffTree_(self):
        #Initialize all payoff nodes to zeros
        self.PayoffTree = np.zeros((self.n+1,self.n+1))
        iopt = 0
        if self.IsCall:
            iopt = 1
        else:
            iopt = -1
        # Get payoffs at each node
        for j in range(self.n+1):
            for i in range(j+1):
                self.PayoffTree[i][j] = np.maximum(0, iopt * (self.StockTree[i][j]-self.K))
        
    def _BackwardsDiscount_(self):
        if self.IsEuropean:
            TerminalPayoff = 0.0
            for i in range(self.n+1):
                TerminalPayoff = TerminalPayoff + self.PayoffTree[i][self.n] * self.ProbTree[i][self.n]
            self.optionprice = TerminalPayoff * np.exp(-self.r * self.T)
        else:
        # Step backward through tree
            if self.IsCall:
                iopt = 1
            else:
                iopt = -1
            for j in range(self.n-1,-1,-1): 
                for i in range(j+1):
                    self.PayoffTree[i][j] = (self.pu * self.PayoffTree[i+1][j+1] \
                                            + self.pd * self.PayoffTree[i][j+1]) * np.exp(-1 * (self.r - self.div) * self.dt)
                    # Use Early exercise price
                    self.PayoffTree[i][j] = np.maximum(iopt * (self.StockTree[i][j] - self.K), self.PayoffTree[i][j])  
            self.optionprice = self.PayoffTree[0][0]
            
    def _BlackScholesOptionPrice_(self):    
        # Computed values for Black Scholes model
        d1 = (np.log(self.S0 / self.K) + (((self.r - self.div) + 0.5 * self.sigma**2) * self.T)) \
                    / (self.sigma * np.sqrt(self.T)) # d1
        d2 = (np.log(self.S0 / self.K) + (((self.r - self.div) - 0.5 * self.sigma**2) * self.T)) \
                    / (self.sigma * np.sqrt(self.T)) # d2
        Nd1 = stats.norm.cdf(d1,0.0,1.0) # N(d1)
        Nd2 = stats.norm.cdf(d2,0.0,1.0) # N(d1)
        Nminusd1 = stats.norm.cdf(-d1,0.0,1.0) # N(-d1)
        Nminusd2 = stats.norm.cdf(-d2,0.0,1.0) # N(-d2)
        
        if self.IsCall: 
            self.BSEuroCalloptionprice = round(self.S0 * np.exp(-self.div * self.T) * Nd1 - \
                self.K * np.exp(-self.r * self.T) * Nd2,4)
        else:
            self.BSEuroPutoptionprice = round(self.K * np.exp(-self.r * self.T) * Nminusd2 - \
                self.S0 * np.exp(-self.div * self.T) * Nminusd1,4)
            
    def _TheGreeks_(self):
        # The Greeks from revaluation after bumps in S0, sigma, r, T
        self.CalcGreeks = 1
        self._CalcDelta_()
        self._CalcVega_()
        self._CalcTheta_()
        self._CalcRho_()
        self._CalcGamma_()
        self._OutputModelGreeks_()
        self.CalcGreeks = 0
        
    def _CalcDelta_(self):
        szero = self.S0
        self.S0 = self.S0 + 1
        # European Call Delta
        self.IsCall, self.IsEuropean = 1, 1
        price = self.BSEuroCalloptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroCallDelta = self.BSEuroCalloptionprice - price
        self.BSEuroCalloptionprice = price
        
        price = self.CRREuroCalloptionprice
        self._CRROptionPrice_()
        self.CRREuroCallDelta = self.CRREuroCalloptionprice - price
        self.CRREuroCalloptionprice = price
        
        price = self.JREuroCalloptionprice
        self._JROptionPrice_()
        self.JREuroCallDelta = self.JREuroCalloptionprice - price
        self.JREuroCalloptionprice = price
        
        price = self.LREuroCalloptionprice
        self._LROptionPrice_()
        self.LREuroCallDelta = self.LREuroCalloptionprice - price
        self.LREuroCalloptionprice = price
        
        # European Put Delta
        self.IsCall, self.IsEuropean = 0, 1
        price = self.BSEuroPutoptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroPutDelta = self.BSEuroPutoptionprice - price
        self.BSEuroPutoptionprice = price
        
        price = self.CRREuroPutoptionprice
        self._CRROptionPrice_()
        self.CRREuroPutDelta = self.CRREuroPutoptionprice - price
        self.CRREuroPutoptionprice = price
        
        price = self.JREuroPutoptionprice
        self._JROptionPrice_()
        self.JREuroPutDelta = self.JREuroPutoptionprice - price
        self.JREuroPutoptionprice = price
        
        price = self.LREuroPutoptionprice
        self._LROptionPrice_()
        self.LREuroPutDelta = self.LREuroPutoptionprice - price
        self.LREuroPutoptionprice = price
        
        # American Call Delta
        self.IsCall, self.IsEuropean = 1, 0
        price = self.CRRAmerCalloptionprice
        self._CRROptionPrice_()
        self.CRRAmerCallDelta = self.CRRAmerCalloptionprice - price
        self.CRRAmerCalloptionprice = price
        
        price = self.JRAmerCalloptionprice
        self._JROptionPrice_()
        self.JRAmerCallDelta = self.JRAmerCalloptionprice - price
        self.JRAmerCalloptionprice = price
        
        price = self.LRAmerCalloptionprice
        self._LROptionPrice_()
        self.LRAmerCallDelta = self.LRAmerCalloptionprice - price
        self.LRAmerCalloptionprice = price
        
        # American Put Delta
        self.IsCall, self.IsEuropean = 0, 0
        price = self.CRRAmerPutoptionprice
        self._CRROptionPrice_()
        self.CRRAmerPutDelta = self.CRRAmerPutoptionprice - price
        self.CRRAmerPutoptionprice = price
        
        price = self.JRAmerPutoptionprice
        self._JROptionPrice_()
        self.JRAmerPutDelta = self.JRAmerPutoptionprice - price
        self.JRAmerPutoptionprice = price
        
        price = self.LRAmerPutoptionprice
        self._LROptionPrice_()
        self.LRAmerPutDelta = self.LRAmerPutoptionprice - price
        self.LRAmerPutoptionprice = price
        self.S0 = szero
     
    def _CalcGamma_(self):
        szero = self.S0
        
        # European Call Gamma
        self.IsCall, self.IsEuropean = 1, 1
        price0 = self.BSEuroCalloptionprice
        delta1 = self.BSEuroCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._BlackScholesOptionPrice_()
        price10 = self.BSEuroCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._BlackScholesOptionPrice_()
        price11 = self.BSEuroCalloptionprice
        delta2 = price11 - price10
        self.BSEuroCallGamma = delta2 - delta1
        self.BSEuroCalloptionprice = price0
        
        price0 = self.CRREuroCalloptionprice
        delta1 = self.CRREuroCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._CRROptionPrice_()
        price10 = self.CRREuroCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._CRROptionPrice_()
        price11 = self.CRREuroCalloptionprice
        delta2 = price11 - price10
        self.CRREuroCallGamma = delta2 - delta1
        self.CRREuroCalloptionprice = price0
        
        price0 = self.JREuroCalloptionprice
        delta1 = self.JREuroCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._JROptionPrice_()
        price10 = self.JREuroCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._JROptionPrice_()
        price11 = self.JREuroCalloptionprice
        delta2 = price11 - price10
        self.JREuroCallGamma = delta2 - delta1
        self.JREuroCalloptionprice = price0
        
        price0 = self.LREuroCalloptionprice
        delta1 = self.LREuroCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._LROptionPrice_()
        price10 = self.LREuroCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._LROptionPrice_()
        price11 = self.LREuroCalloptionprice
        delta2 = price11 - price10
        self.LREuroCallGamma = delta2 - delta1
        self.LREuroCalloptionprice = price0
        
        # European Put Gamma
        self.IsCall, self.IsEuropean = 0, 1
        price0 = self.BSEuroPutoptionprice
        delta1 = self.BSEuroPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._BlackScholesOptionPrice_()
        price10 = self.BSEuroPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._BlackScholesOptionPrice_()
        price11 = self.BSEuroPutoptionprice
        delta2 = price11 - price10
        self.BSEuroPutGamma = delta2 - delta1
        self.BSEuroPutoptionprice = price0
        
        price0 = self.CRREuroPutoptionprice
        delta1 = self.CRREuroPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._CRROptionPrice_()
        price10 = self.CRREuroPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._CRROptionPrice_()
        price11 = self.CRREuroPutoptionprice
        delta2 = price11 - price10
        self.CRREuroPutGamma = delta2 - delta1
        self.CRREuroPutoptionprice = price0
        
        price0 = self.JREuroPutoptionprice
        delta1 = self.JREuroPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._JROptionPrice_()
        price10 = self.JREuroPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._JROptionPrice_()
        price11 = self.JREuroPutoptionprice
        delta2 = price11 - price10
        self.JREuroPutGamma = delta2 - delta1
        self.JREuroPutoptionprice = price0
        
        price0 = self.LREuroPutoptionprice
        delta1 = self.LREuroPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._LROptionPrice_()
        price10 = self.LREuroPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._LROptionPrice_()
        price11 = self.LREuroPutoptionprice
        delta2 = price11 - price10
        self.LREuroPutGamma = delta2 - delta1
        self.LREuroPutoptionprice = price0
        
        # American Call Gamma
        self.IsCall, self.IsEuropean = 1, 0
        price0 = self.CRRAmerCalloptionprice
        delta1 = self.CRRAmerCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._CRROptionPrice_()
        price10 = self.CRRAmerCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._CRROptionPrice_()
        price11 = self.CRRAmerCalloptionprice
        delta2 = price11 - price10
        self.CRRAmerCallGamma = delta2 - delta1
        self.CRRAmerCalloptionprice = price0
        
        price0 = self.JRAmerCalloptionprice
        delta1 = self.JRAmerCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._JROptionPrice_()
        price10 = self.JRAmerCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._JROptionPrice_()
        price11 = self.JRAmerCalloptionprice
        delta2 = price11 - price10
        self.JRAmerCallGamma = delta2 - delta1
        self.JRAmerCalloptionprice = price0
        
        price0 = self.LRAmerCalloptionprice
        delta1 = self.LRAmerCallDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._LROptionPrice_()
        price10 = self.LRAmerCalloptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._LROptionPrice_()
        price11 = self.LRAmerCalloptionprice
        delta2 = price11 - price10
        self.LRAmerCallGamma = delta2 - delta1
        self.LRAmerCalloptionprice = price0
        
        # American Put Gamma
        self.IsCall, self.IsEuropean = 0, 0
        price0 = self.CRRAmerPutoptionprice
        delta1 = self.CRRAmerPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._CRROptionPrice_()
        price10 = self.CRRAmerPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._CRROptionPrice_()
        price11 = self.CRRAmerPutoptionprice
        delta2 = price11 - price10
        self.CRRAmerPutGamma = delta2 - delta1
        self.CRRAmerPutoptionprice = price0
        
        price0 = self.JRAmerPutoptionprice
        delta1 = self.JRAmerPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._JROptionPrice_()
        price10 = self.JRAmerPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._JROptionPrice_()
        price11 = self.JRAmerPutoptionprice
        delta2 = price11 - price10
        self.JRAmerPutGamma = delta2 - delta1
        self.JRAmerPutoptionprice = price0
        
        price0 = self.LRAmerPutoptionprice
        delta1 = self.LRAmerPutDelta
        self.S0 = self.S0 + 10 # add 10 dollars
        self._LROptionPrice_()
        price10 = self.LRAmerPutoptionprice
        self.S0 = self.S0 + 1 # add one more dollar
        self._LROptionPrice_()
        price11 = self.LRAmerPutoptionprice
        delta2 = price11 - price10
        self.LRAmerPutGamma = delta2 - delta1
        self.LRAmerPutoptionprice = price0
        self.S0 = szero
        
        
    def _CalcVega_(self):
        sigmazero = self.sigma
        self.sigma = self.sigma + .01
        # European Call Vega
        self.IsCall, self.IsEuropean = 1, 1
        price = self.BSEuroCalloptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroCallVega = self.BSEuroCalloptionprice - price
        self.BSEuroCalloptionprice = price
        
        price = self.CRREuroCalloptionprice
        self._CRROptionPrice_()
        self.CRREuroCallVega = self.CRREuroCalloptionprice - price
        self.CRREuroCalloptionprice = price
        
        price = self.JREuroCalloptionprice
        self._JROptionPrice_()
        self.JREuroCallVega = self.JREuroCalloptionprice - price
        self.JREuroCalloptionprice = price
        
        price = self.LREuroCalloptionprice
        self._LROptionPrice_()
        self.LREuroCallVega = self.LREuroCalloptionprice - price
        self.LREuroCalloptionprice = price
        
        # European Put Vega
        self.IsCall, self.IsEuropean = 0, 1
        price = self.BSEuroPutoptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroPutVega = self.BSEuroPutoptionprice - price
        self.BSEuroPutoptionprice = price
        
        price = self.CRREuroPutoptionprice
        self._CRROptionPrice_()
        self.CRREuroPutVega = self.CRREuroPutoptionprice - price
        self.CRREuroPutoptionprice = price
        
        price = self.JREuroPutoptionprice
        self._JROptionPrice_()
        self.JREuroPutVega = self.JREuroPutoptionprice - price
        self.JREuroPutoptionprice = price
        
        price = self.LREuroPutoptionprice
        self._LROptionPrice_()
        self.LREuroPutVega = self.LREuroPutoptionprice - price
        self.LREuroPutoptionprice = price
        
        # American Call Vega
        self.IsCall, self.IsEuropean = 1, 0
        price = self.CRRAmerCalloptionprice
        self._CRROptionPrice_()
        self.CRRAmerCallVega = self.CRRAmerCalloptionprice - price
        self.CRRAmerCalloptionprice = price
        
        price = self.JRAmerCalloptionprice
        self._JROptionPrice_()
        self.JRAmerCallVega = self.JRAmerCalloptionprice - price
        self.JRAmerCalloptionprice = price
        
        price = self.LRAmerCalloptionprice
        self._LROptionPrice_()
        self.LRAmerCallVega = self.LRAmerCalloptionprice - price
        self.LRAmerCalloptionprice = price
        
        # American Put Vega
        self.IsCall, self.IsEuropean = 0, 0
        price = self.CRRAmerPutoptionprice
        self._CRROptionPrice_()
        self.CRRAmerPutVega = self.CRRAmerPutoptionprice - price
        self.CRRAmerPutoptionprice = price
        
        price = self.JRAmerPutoptionprice
        self._JROptionPrice_()
        self.JRAmerPutVega = self.JRAmerPutoptionprice - price
        self.JRAmerPutoptionprice = price
        
        price = self.LRAmerPutoptionprice
        self._LROptionPrice_()
        self.LRAmerPutVega = self.LRAmerPutoptionprice - price
        self.LRAmerPutoptionprice = price
        self.sigma = sigmazero
        
    def _CalcTheta_(self):
        tzero = self.T
        self.T = self.T - 1/365
        # European Call Theta
        self.IsCall, self.IsEuropean = 1, 1
        price = self.BSEuroCalloptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroCallTheta = self.BSEuroCalloptionprice - price
        self.BSEuroCalloptionprice = price
        
        price = self.CRREuroCalloptionprice
        self._CRROptionPrice_()
        self.CRREuroCallTheta = self.CRREuroCalloptionprice - price
        self.CRREuroCalloptionprice = price
        
        price = self.JREuroCalloptionprice
        self._JROptionPrice_()
        self.JREuroCallTheta = self.JREuroCalloptionprice - price
        self.JREuroCalloptionprice = price
        
        price = self.LREuroCalloptionprice
        self._LROptionPrice_()
        self.LREuroCallTheta = self.LREuroCalloptionprice - price
        self.LREuroCalloptionprice = price
        
        # European Put Theta
        self.IsCall, self.IsEuropean = 0, 1
        price = self.BSEuroPutoptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroPutTheta = self.BSEuroPutoptionprice - price
        self.BSEuroPutoptionprice = price
        
        price = self.CRREuroPutoptionprice
        self._CRROptionPrice_()
        self.CRREuroPutTheta = self.CRREuroPutoptionprice - price
        self.CRREuroPutoptionprice = price
        
        price = self.JREuroPutoptionprice
        self._JROptionPrice_()
        self.JREuroPutTheta = self.JREuroPutoptionprice - price
        self.JREuroPutoptionprice = price
        
        price = self.LREuroPutoptionprice
        self._LROptionPrice_()
        self.LREuroPutTheta = self.LREuroPutoptionprice - price
        self.LREuroPutoptionprice = price
        
        # American Call Theta
        self.IsCall, self.IsEuropean = 1, 0
        price = self.CRRAmerCalloptionprice
        self._CRROptionPrice_()
        self.CRRAmerCallTheta = self.CRRAmerCalloptionprice - price
        self.CRRAmerCalloptionprice = price
        
        price = self.JRAmerCalloptionprice
        self._JROptionPrice_()
        self.JRAmerCallTheta = self.JRAmerCalloptionprice - price
        self.JRAmerCalloptionprice = price
        
        price = self.LRAmerCalloptionprice
        self._LROptionPrice_()
        self.LRAmerCallTheta = self.LRAmerCalloptionprice - price
        self.LRAmerCalloptionprice = price
        
        # American Put Theta
        self.IsCall, self.IsEuropean = 0, 0
        price = self.CRRAmerPutoptionprice
        self._CRROptionPrice_()
        self.CRRAmerPutTheta = self.CRRAmerPutoptionprice - price
        self.CRRAmerPutoptionprice = price
        
        price = self.JRAmerPutoptionprice
        self._JROptionPrice_()
        self.JRAmerPutTheta = self.JRAmerPutoptionprice - price
        self.JRAmerPutoptionprice = price
        
        price = self.LRAmerPutoptionprice
        self._LROptionPrice_()
        self.LRAmerPutTheta = self.LRAmerPutoptionprice - price
        self.LRAmerPutoptionprice = price
        self.T = tzero
            
    def _CalcRho_(self):
        rzero = self.r
        self.r = self.r + 0.01
        # European Call Rho
        self.IsCall, self.IsEuropean = 1, 1
        price = self.BSEuroCalloptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroCallRho = self.BSEuroCalloptionprice - price
        self.BSEuroCalloptionprice = price
        
        price = self.CRREuroCalloptionprice
        self._CRROptionPrice_()
        self.CRREuroCallRho = self.CRREuroCalloptionprice - price
        self.CRREuroCalloptionprice = price
        
        price = self.JREuroCalloptionprice
        self._JROptionPrice_()
        self.JREuroCallRho = self.JREuroCalloptionprice - price
        self.JREuroCalloptionprice = price
        
        price = self.LREuroCalloptionprice
        self._LROptionPrice_()
        self.LREuroCallRho = self.LREuroCalloptionprice - price
        self.LREuroCalloptionprice = price
        
        # European Put Rho
        self.IsCall, self.IsEuropean = 0, 1
        price = self.BSEuroPutoptionprice
        self._BlackScholesOptionPrice_()
        self.BSEuroPutRho = self.BSEuroPutoptionprice - price
        self.BSEuroPutoptionprice = price
        
        price = self.CRREuroPutoptionprice
        self._CRROptionPrice_()
        self.CRREuroPutRho = self.CRREuroPutoptionprice - price
        self.CRREuroPutoptionprice = price
        
        price = self.JREuroPutoptionprice
        self._JROptionPrice_()
        self.JREuroPutRho = self.JREuroPutoptionprice - price
        self.JREuroPutoptionprice = price
        
        price = self.LREuroPutoptionprice
        self._LROptionPrice_()
        self.LREuroPutRho = self.LREuroPutoptionprice - price
        self.LREuroPutoptionprice = price
        
        # American Call Rho
        self.IsCall, self.IsEuropean = 1, 0
        price = self.CRRAmerCalloptionprice
        self._CRROptionPrice_()
        self.CRRAmerCallRho = self.CRRAmerCalloptionprice - price
        self.CRRAmerCalloptionprice = price
        
        price = self.JRAmerCalloptionprice
        self._JROptionPrice_()
        self.JRAmerCallRho = self.JRAmerCalloptionprice - price
        self.JRAmerCalloptionprice = price
        
        price = self.LRAmerCalloptionprice
        self._LROptionPrice_()
        self.LRAmerCallRho = self.LRAmerCalloptionprice - price
        self.LRAmerCalloptionprice = price
        
        # American Put Rho
        self.IsCall, self.IsEuropean = 0, 0
        price = self.CRRAmerPutoptionprice
        self._CRROptionPrice_()
        self.CRRAmerPutRho = self.CRRAmerPutoptionprice - price
        self.CRRAmerPutoptionprice = price
        
        price = self.JRAmerPutoptionprice
        self._JROptionPrice_()
        self.JRAmerPutRho = self.JRAmerPutoptionprice - price
        self.JRAmerPutoptionprice = price
        
        price = self.LRAmerPutoptionprice
        self._LROptionPrice_()
        self.LRAmerPutRho = self.LRAmerPutoptionprice - price
        self.LRAmerPutoptionprice = price
        self.r = rzero
        
    def _OutputModelPrices_(self):
        # Output model price to Excel file
        self.ws_models['E5'] = self.BSEuroCalloptionprice
        self.ws_models['E6'] = self.CRREuroCalloptionprice
        self.ws_models['E7'] = self.JREuroCalloptionprice
        self.ws_models['E8'] = self.LREuroCalloptionprice

        self.ws_models['K5'] = self.BSEuroPutoptionprice
        self.ws_models['K6'] = self.CRREuroPutoptionprice
        self.ws_models['K7'] = self.JREuroPutoptionprice
        self.ws_models['K8'] = self.LREuroPutoptionprice

        self.ws_models['E12'] = self.CRRAmerCalloptionprice
        self.ws_models['E13'] = self.JRAmerCalloptionprice
        self.ws_models['E14'] = self.LRAmerCalloptionprice
        
        self.ws_models['K12'] = self.CRRAmerPutoptionprice
        self.ws_models['K13'] = self.JRAmerPutoptionprice
        self.ws_models['K14'] = self.LRAmerPutoptionprice
    
    def _OutputModelTrees_(self):
        # Output model trees to Excel file
        model = self.model
        if self.IsCall and self.IsEuropean:
            self.ws_trees = self.wb_models.create_sheet(model + 'EuroCall')
        elif not self.IsCall and self.IsEuropean:
            self.ws_trees = self.wb_models.create_sheet(model + 'EuroPut')
        elif self.IsCall and not self.IsEuropean:
            self.ws_trees = self.wb_models.create_sheet(model + 'AmerCall')
        else:
            self.ws_trees = self.wb_models.create_sheet(model + 'AmerPut')

        self.ws_trees.cell(1,1,str(model) + ' Stock Tree :: ' + str(self.n) + ' Time Steps')
        for j in range(self.n+1):
            for i in range(j+1):
                if j == 1:
                    for k in range(self.n+1):
                        self.ws_trees.cell(self.n+1-k+1,j,k)
                self.ws_trees.cell(self.n+1-i+1,j+2,round(self.StockTree[i][j],4))
                self.ws_trees.cell(self.n+3,j+2,j)
        
        self.ws_trees.cell(self.n+5,1, str(model) + ' Probability Tree :: ' + str(self.n) + ' Time Steps')
        for j in range(self.n+1):
            for i in range(j+1):
                if j == 1:
                    for k in range(self.n+1):
                        self.ws_trees.cell(2*(self.n)+5-k+1,j,k)
                self.ws_trees.cell(2*(self.n)+5-i+1,j+2,round(self.ProbTree[i][j],4))
                self.ws_trees.cell(2*(self.n)+7,j+2,j)

        self.ws_trees.cell(2*(self.n)+9,1,str(model) + ' Payoff Tree :: ' + str(self.n) + ' Time Steps')
        for j in range(self.n+1):
            for i in range(j+1):
                if j == 1:
                    for k in range(self.n+1):
                        self.ws_trees.cell(3*(self.n)+9-k+1,j,k)
                self.ws_trees.cell(3*(self.n)+9-i+1,j+2,round(self.PayoffTree[i][j],4))
                self.ws_trees.cell(3*(self.n)+11,j+2,j)
                
                
    def _OutputModelGreeks_(self):
        # Output model price to Excel file
        # European Call Delta
        self.ws_models['F5'] = self.BSEuroCallDelta
        self.ws_models['F6'] = self.CRREuroCallDelta
        self.ws_models['F7'] = self.JREuroCallDelta
        self.ws_models['F8'] = self.LREuroCallDelta
        # European Put Delta
        self.ws_models['L5'] = self.BSEuroPutDelta
        self.ws_models['L6'] = self.CRREuroPutDelta
        self.ws_models['L7'] = self.JREuroPutDelta
        self.ws_models['L8'] = self.LREuroPutDelta
        # American Call Delta
        self.ws_models['F12'] = self.CRRAmerCallDelta
        self.ws_models['F13'] = self.JRAmerCallDelta
        self.ws_models['F14'] = self.LRAmerCallDelta
        # American Put Delta
        self.ws_models['L12'] = self.CRRAmerPutDelta
        self.ws_models['L13'] = self.JRAmerPutDelta
        self.ws_models['L14'] = self.LRAmerPutDelta
        
        # European Call Gamma
        self.ws_models['G5'] = self.BSEuroCallGamma
        self.ws_models['G6'] = self.CRREuroCallGamma
        self.ws_models['G7'] = self.JREuroCallGamma
        self.ws_models['G8'] = self.LREuroCallGamma
        # European Put Gamma
        self.ws_models['M5'] = self.BSEuroPutGamma
        self.ws_models['M6'] = self.CRREuroPutGamma
        self.ws_models['M7'] = self.JREuroPutGamma
        self.ws_models['M8'] = self.LREuroPutGamma
        # American Call Gamma
        self.ws_models['G12'] = self.CRRAmerCallGamma
        self.ws_models['G13'] = self.JRAmerCallGamma
        self.ws_models['G14'] = self.LRAmerCallGamma
        # American Put Gamma
        self.ws_models['M12'] = self.CRRAmerPutGamma
        self.ws_models['M13'] = self.JRAmerPutGamma
        self.ws_models['M14'] = self.LRAmerPutGamma
        
        # European Call Vega
        self.ws_models['H5'] = self.BSEuroCallVega
        self.ws_models['H6'] = self.CRREuroCallVega
        self.ws_models['H7'] = self.JREuroCallVega
        self.ws_models['H8'] = self.LREuroCallVega
        # European Put Vega
        self.ws_models['N5'] = self.BSEuroPutVega
        self.ws_models['N6'] = self.CRREuroPutVega
        self.ws_models['N7'] = self.JREuroPutVega
        self.ws_models['N8'] = self.LREuroPutVega
        # American Call Vega
        self.ws_models['H12'] = self.CRRAmerCallVega
        self.ws_models['H13'] = self.JRAmerCallVega
        self.ws_models['H14'] = self.LRAmerCallVega
        # American Put Vega
        self.ws_models['N12'] = self.CRRAmerPutVega
        self.ws_models['N13'] = self.JRAmerPutVega
        self.ws_models['N14'] = self.LRAmerPutVega
        
        # European Call Theta
        self.ws_models['I5'] = self.BSEuroCallTheta
        self.ws_models['I6'] = self.CRREuroCallTheta
        self.ws_models['I7'] = self.JREuroCallTheta
        self.ws_models['I8'] = self.LREuroCallTheta
        # European Put Theta
        self.ws_models['O5'] = self.BSEuroPutTheta
        self.ws_models['O6'] = self.CRREuroPutTheta
        self.ws_models['O7'] = self.JREuroPutTheta
        self.ws_models['O8'] = self.LREuroPutTheta
        # American Call Theta
        self.ws_models['I12'] = self.CRRAmerCallTheta
        self.ws_models['I13'] = self.JRAmerCallTheta
        self.ws_models['I14'] = self.LRAmerCallTheta
        # American Put Theta
        self.ws_models['O12'] = self.CRRAmerPutTheta
        self.ws_models['O13'] = self.JRAmerPutTheta
        self.ws_models['O14'] = self.LRAmerPutTheta
        
        # European Call Rho
        self.ws_models['J5'] = self.BSEuroCallRho
        self.ws_models['J6'] = self.CRREuroCallRho
        self.ws_models['J7'] = self.JREuroCallRho
        self.ws_models['J8'] = self.LREuroCallRho
        # European Put Rho
        self.ws_models['P5'] = self.BSEuroPutRho
        self.ws_models['P6'] = self.CRREuroPutRho
        self.ws_models['P7'] = self.JREuroPutRho
        self.ws_models['P8'] = self.LREuroPutRho
        # American Call Rho
        self.ws_models['J12'] = self.CRRAmerCallRho
        self.ws_models['J13'] = self.JRAmerCallRho
        self.ws_models['J14'] = self.LRAmerCallRho
        # American Put Rho
        self.ws_models['P12'] = self.CRRAmerPutRho
        self.ws_models['P13'] = self.JRAmerPutRho
        self.ws_models['P14'] = self.LRAmerPutRho
        
    def PriceAll(self):
        self.IsCall = 1
        self.IsEuropean = 1
        
        while self.IsCall and self.IsEuropean:
            self._BlackScholesOptionPrice_()
            self._CRROptionPrice_()
            self._JROptionPrice_()
            self._LROptionPrice_()
            self.IsCall = 0
            
        while not self.IsCall and self.IsEuropean:
            self._BlackScholesOptionPrice_()
            self._CRROptionPrice_()
            self._JROptionPrice_()
            self._LROptionPrice_()
            self.IsCall = 1
            self.IsEuropean = 0
            
        while self.IsCall and not self.IsEuropean:
            self._CRROptionPrice_()
            self._JROptionPrice_()
            self._LROptionPrice_()
            self.IsCall = 0
        
        while not self.IsCall and not self.IsEuropean:
            self._CRROptionPrice_()
            self._JROptionPrice_()
            self._LROptionPrice_()
            break
        self._OutputModelPrices_()
        self.wb_models.save('stockoptiondata.xlsx')
        self._TheGreeks_()
        
        # Save your file to commit your calculations from memory to your Excel file
        # Your Excel file must be closed or you will get 'Permission Denied' to write
        self.wb_models.save('stockoptiondata.xlsx')
    
if __name__ == "__main__":
    
    binomialoption = BinomialOption()
    binomialoption.PriceAll()
    